#  第6章 数据加载、存储与⽂件格式 Data Loading, Storage, 

* 访问数据是使⽤本书所介绍的这些⼯具的第⼀步。我会着重介绍pandas的数据输⼊与输出，虽然别的库中也有不少以此为⽬的的⼯具

* 输⼊输出通常可以划分为⼏个⼤类：读取⽂本⽂件和其他更⾼效
的磁盘存储格式，加载数据库中的数据，利⽤Web API操作⽹络
资源。

In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

##  6.1 读写⽂本格式的数据 Reading and Writing Data in Text Format

In [30]:
!type  G:\test\pydata-book\examples\ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [31]:
df = pd.read_csv('G:/test/pydata-book/examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [32]:
# 我们还可以使⽤read_table，并指定分隔符：
pd.read_table('G:/test/pydata-book/examples/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 并不是所有⽂件都有标题⾏。看看下⾯这个⽂件

In [33]:
!type  G:\test\pydata-book\examples\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


* 读⼊该⽂件的办法有两个。你可以让pandas为其分配默认的列名，也可以⾃⼰定义列名

In [37]:
pd.read_csv('G:/test/pydata-book/examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [38]:
pd.read_csv('G:/test/pydata-book/examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 假设你希望将message列做成DataFrame的索引。你可以明确表示要将该列放到索引4的位置上，也可以通过index_col参数指定"message"：

In [39]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('G:/test/pydata-book/examples/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


* 如果希望将多个列做成⼀个层次化索引，只需传⼊由列编号或列名组成的列表即可：

In [41]:
!type  G:\test\pydata-book\examples\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [42]:
parsed = pd.read_csv('G:/test/pydata-book/examples/csv_mindex.csv',
                     index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [ ]:
* 有些情况下，有些表格可能不是⽤固定的分隔符去分隔字段的
（⽐如空⽩符或其他模式）。有些表格可能不是⽤固定的分隔符
去分隔字段的（⽐如空⽩符或其他模式来分隔字段）。看看下⾯
这个⽂本⽂件：

In [45]:
list(open('G:/test/pydata-book/examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

* 虽然可以⼿动对数据进⾏规整，这⾥的字段是被数量不同的空⽩字符间隔开的。这种情况下，你可以传递⼀个正则表达式作为read_table的分隔符。可以⽤正则表达式表达为\s+，于是有有：

In [46]:
result = pd.read_table('G:/test/pydata-book/examples/ex3.txt', sep='\s+')
resultbb

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


* 这⾥，由于列名⽐数据⾏的数量少，所以read_table推断第⼀列应该是DataFrame的索引。这⾥，由于列名⽐数据⾏的数量少，所以read_table推断第⼀列应该是DataFrame的索引。这些解析器函数还有许多参数可以帮助你处理各种各样的异形件格式。⽐如说，你可以⽤skiprows跳过⽂件的第⼀⾏、第三⾏和第四⾏：

In [49]:
!type G:\test\pydata-book\examples\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [ ]:
pd.read_csv('G:/test/pydata-book/examples/ex4.csv', skiprows=[0, 2, 3])

* 缺失值处理是⽂件解析任务中的⼀个重要组成部分。缺失数据经常是要么没有（空字符串），要么⽤某个标记值表示。默认情况下， pandas会⽤⼀组经常出现的标记值进⾏识别，⽐如NA及NULL：

In [51]:
!type G:\test\pydata-book\examples\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [52]:
result = pd.read_csv('G:/test/pydata-book/examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [53]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


* na_values可以⽤⼀个列表或集合的字符串表示缺失值：

In [54]:
result = pd.read_csv('G:/test/pydata-book/examples/ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


* 字典的各列可以使⽤不同的NA标记值：

In [55]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('G:/test/pydata-book/examples/ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### 逐块读取⽂本⽂件 Reading Text Files in Pieces

* 在处理很⼤的⽂件时，或找出⼤⽂件中的参数集以便于后续处理
时，你可能只想读取⽂件的⼀⼩部分或逐块对⽂件进⾏迭代。
在看⼤⽂件之前，我们先设置pandas显示地更紧些：

In [57]:
pd.options.display.max_rows = 10

In [58]:
result = pd.read_csv('G:/test/pydata-book/examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


* 如果只想读取⼏⾏（避免读取整个⽂件），通过nrows进⾏指定
即可

In [59]:
pd.read_csv('G:/test/pydata-book/examples/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


* 要逐块读取⽂件，可以指定chunksize（⾏数）：

In [60]:
chunker = pd.read_csv('G:/test/pydata-book/examples/ex6.csv', chunksize=1000)
chunker

* read_csv所返回的这个TextParser对象使你可以根据chunksize对⽂件进⾏逐块迭代。⽐如说，我们可以迭代处理ex6.csv，将值计数聚合到"key"列中，如下所示：

In [61]:
chunker = pd.read_csv('G:/test/pydata-book/examples/ex6.csv', chunksize=1000)

tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

* 然后有

In [62]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

* TextParser还有⼀个get_chunk⽅法，它使你可以读取任意⼤⼩的块。

###   将数据写出到⽂本格式 Writing Data to Text Format

In [63]:
data = pd.read_csv('G:/test/pydata-book/examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


* 利⽤DataFrame的to_csv⽅法，我们可以将数据写到⼀个以逗号分隔的⽂件中：

In [64]:
data.to_csv('G:/test/pydata-book/examples/out.csv')
!type G:\test\pydata-book\examples\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


* 当然，还可以使⽤其他分隔符（由于这⾥直接写出到
sys.stdout，所以仅仅是打印出⽂本结果⽽已）：

In [65]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


* 缺失值在输出结果中会被表示为空字符串。你可能希望将其表示为别的标记值：

In [66]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


* 如果没有设置其他选项，则会写出⾏和列的标签。当然，它们也
都可以被禁⽤：

In [67]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


* 此外，你还可以只写出⼀部分的列，并以你指定的顺序排列：

In [68]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [70]:
dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('G:/test/pydata-book/examples/tseries.csv')
!type G:\test\pydata-book\examples\tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


###  处理分隔符格式 Working with Delimited Formats

* ⼤部分存储在磁盘上的表格型数据都能⽤pandas.read_table进
⾏加载。然⽽，有时还是需要做⼀些⼿⼯处理。由于接收到含有
畸形⾏的⽂件⽽使read_table出⽑病的情况并不少⻅。为了说明
这些基本⼯具，看看下⾯这个简单的CSV⽂件：

In [71]:
!type  G:\test\pydata-book\examples\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [ ]:
* 对于任何单字符分隔符⽂件，可以直接使⽤Python内置的csv模
块。将任意已打开的⽂件或⽂件型的对象传给csv.reader：

In [72]:
import csv
f = open('G:/test/pydata-book/examples/ex7.csv')

reader = csv.reader(f)

* 对这个reader进⾏迭代将会为每⾏产⽣⼀个元组（并移除了所有
的引号）：对这个reader进⾏迭代将会为每⾏产⽣⼀个元组（并
移除了所有的引号）：

In [73]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


* 现在，为了使数据格式合乎要求，你需要对其做⼀些整理⼯作。
我们⼀步⼀步来做。⾸先，读取⽂件到⼀个多⾏的列表中：

In [74]:
with open('G:/test/pydata-book/examples/ex7.csv') as f:
    lines = list(csv.reader(f))

* 然后，我们将这些⾏分为标题⾏和数据⾏：

In [75]:
header, values = lines[0], lines[1:]

* 然后，我们可以⽤字典构造式和zip(*values)，后者将⾏转置为
列，创建数据列的字典：

In [76]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

* CSV⽂件的形式有很多。只需定义csv.Dialect的⼀个⼦类即可定
义出新格式（如专⻔的分隔符、字符串引⽤约定、⾏结束符等）：

In [77]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

* 各个CSV语⽀的参数也可以关键字的形式提供给csv.reader，⽽
⽆需定义⼦类：

In [ ]:
reader = csv.reader(f, dialect=my_dialect)a

In [ ]:
reader = csv.reader(f, delimiter='|')

*  对于那些使⽤复杂分隔符或多字符分隔符的⽂件， csv
模块就⽆能为⼒了。这种情况下，你就只能使⽤字符串的split
⽅法或正则表达式⽅法re.split进⾏⾏拆分和其他整理⼯作
了 

* 要⼿⼯输出分隔符⽂件，你可以使⽤csv.writer。它接受⼀个已打
开且可写的⽂件对象以及跟csv.reader相同的那些语⽀和格式化
选项：

In [81]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

### JSON数据 JSON Data

* JSON（JavaScript Object Notation的简称）已经成为通过HTTP
请求在Web浏览器和其他应⽤程序之间发送数据的标准格式之
⼀。它是⼀种⽐表格型⽂本格式（如CSV）灵活得多的数据格
式。下⾯是⼀个例⼦

In [82]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [83]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

* json.dumps则将Python对象转换成JSON格式

In [84]:
asjson = json.dumps(result)

* 如何将（⼀个或⼀组） JSON对象转换为DataFrame或其他便于
分析的数据结构就由你决定了。最简单⽅便的⽅式是：向
DataFrame构造器传⼊⼀个字典的列表（就是原先的JSON对
象），并选取数据字段的⼦集：

In [85]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


* pandas.read_json可以⾃动将特别格式的JSON数据集转换为
Series或DataFrame。例如

In [87]:
!type  G:\test\pydata-book\examples\example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


* pandas.read_json的默认选项假设JSON数组中的每个对象是表
格中的⼀⾏：

In [88]:
data = pd.read_json('G:/test/pydata-book/examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [89]:
print(data.to_json())
print(data.to_json(orient='records'))

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}
[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


###  XML和HTML： Web信息收集 XML and HTML: Web Scraping

* Python有许多可以读写常⻅的HTML和XML格式数据的库，包括
lxml、 Beautiful Soup和html5lib。 lxml的速度⽐较快，但其它的
库处理有误的HTML或XML⽂件更好。
pandas有⼀个内置的功能， read_html，它可以使⽤lxml和
Beautiful Soup⾃动将HTML⽂件中的表格解析为DataFrame对
象。为了进⾏展示，我从美国联邦存款保险公司下载了⼀个
HTML⽂件（pandas⽂档中也使⽤过），它记录了银⾏倒闭的情
况。⾸先，你需要安装read_html⽤到的库

In [ ]:
conda install lxml
pip install beautifulsoup4 html5lib

In [91]:
tables = pd.read_html('G:/test/pydata-book/examples/fdic_failed_bank_list.html')
len(tables)

1

In [92]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


*  因为failures有许多列， pandas插⼊了⼀个换⾏符\。

* 这⾥，我们可以做⼀些数据清洗和分析（后⾯章节会进⼀步讲
解），⽐如计算按年份计算倒闭的银⾏数：

In [93]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

####    利⽤lxml.objectify解析XML  Parsing XML with lxml.objectify

* XML（Extensible Markup Language）是另⼀种常⻅的⽀持分
层、嵌套数据以及元数据的结构化数据格式。本书所使⽤的这些
⽂件实际上来⾃于⼀个很⼤的XML⽂档。
前⾯，我介绍了pandas.read_html函数，它可以使⽤lxml或
Beautiful Soup从HTML解析数据。 XML和HTML的结构很相似，
danXML更为通⽤。这⾥，我会⽤⼀个例⼦演示如何利⽤lxml从
XML格式解析数据。
纽约⼤都会运输署发布了⼀些有关其公交和列⻋服务的数据资料
（http://www.mta.info/developers/download.html）。这⾥，我
们将看看包含在⼀组XML⽂件中的运⾏情况数据。每项列⻋或公
交服务都有各⾃的⽂件（如Metro-North Railroad的⽂件是
Performance_MNR.xml），其中每条XML记录就是⼀条⽉度数
据，如下所示：

In [ ]:
<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>

 * 我们先⽤lxml.objectify解析该⽂件，然后通过getroot得到该XML
⽂件的根节点的引⽤：

In [96]:
from lxml import objectify

path = 'G:/test/pydata-book/datasets/mta_perf/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

* root.INDICATOR返回⼀个⽤于产⽣各个<INDICATOR>XML元素
的⽣成器。对于每条记录，我们可以⽤标记名（如
YTD_ACTUAL）和数据值填充⼀个字典（排除⼏个标记）

In [97]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

* 最后，将这组字典转换为⼀个DataFrame

In [98]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95


In [99]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

In [100]:
root
root.get('href')
root.text

'Google'

##  6.2 ⼆进制数据格式 Binary Data Formats

* 实现数据的⾼效⼆进制格式存储最简单的办法之⼀是使⽤Python
内置的pickle序列化。 pandas对象都有⼀个⽤于将数据以pickle
格式保存到磁盘上的to_pickle⽅法：

In [101]:
frame = pd.read_csv('G:/test/pydata-book/examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [102]:
frame.to_pickle('G:/test/pydata-book/examples/frame_pickle')

* 你可以通过pickle直接读取被pickle化的数据，或是使⽤更为⽅便
的pandas.read_pickle：

In [103]:
pd.read_pickle('G:/test/pydata-book/examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* pickle仅建议⽤于短期存储格式。其原因是很难保证该
格式永远是稳定的；今天pickle的对象可能⽆法被后续版本的
库unpickle出来。虽然我尽⼒保证这种事情不会发⽣在
pandas中，但是今后的某个时候说不定还是得“打破”该pickle
格式。

In [107]:
!type G:\test\pydata-book\examples\frame_pickle

�晼      �pandas.core.frame攲	DataFrame敁�)仈}�(�	_metadata擼攲_data攲pandas.core.internals攲BlockManager敁�)仈(]�(�pandas.core.indexes.base攲
_new_Index敁攈
�Index敁攠�(�data攲numpy.core.multiarray攲_reconstruct敁攲numpy攲ndarray敁擪 厰Cb攪擱�(KK厰�numpy攲dtype敁攲O8擪 K嚁R�(K�|擭NNJ����J����K?t攂塢�(�a攲b攲c攲d攲message攅t攂�name擭u啍R攈�pandas.core.indexes.range攲
RangeIndex敁攠�(�step擪�stop擪�start擪 h.Nu啍R攅]�(hhK 厰h嚁R�(KKK啍h!�i8擪 K嚁R�(K�<擭NNJ����J����K t攂塁`              	                     
                                                 攖攂hhK 厰h嚁R�(KKK啍h$塢�(�hello攲world攲foo攅t攂e]�(hh}�(hhhK 厰h嚁R�(KK厰h$塢�(h(h)h*h+et攂h.Nu啍R攈h}�(hhhK 厰h嚁R�(KK厰h$塢攈,at攂h.Nu啍R攅}攲0.14.1攠�(�axes攈�blocks擼�(}�(�mgr_locs攲builtins攲slice敁擪 KK嚁R攲values攈=u}�(hihlKKK嚁R攈ohHueust攂�_typ攲	dataframe攗b.


###  使⽤HDF5格式 Using HDF5 Format

* HDF5是⼀种存储⼤规模科学数组数据的⾮常好的⽂件格式。它
可以被作为C库，带有许多语⾔的接⼝，如Java、 Python和
MATLAB等。 HDF5中的HDF指的是层次型数据格式
（hierarchical data format）。每个HDF5⽂件都含有⼀个⽂件系
统式的节点结构，它使你能够存储多个数据集并⽀持元数据。与
其他简单格式相⽐， HDF5⽀持多种压缩器的即时压缩，还能更
⾼效地存储重复模式数据。对于那些⾮常⼤的⽆法直接放⼊内存
的数据集， HDF5就是不错的选择，因为它可以⾼效地分块读
写。
* 虽然可以⽤PyTables或h5py库直接访问HDF5⽂件， pandas提供
了更为⾼级的接⼝，可以简化存储Series和DataFrame对象。
HDFStore类可以像字典⼀样，处理低级的细节：

In [108]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

* HDF5⽂件中的对象可以通过与字典⼀样的API进⾏获取：

In [109]:
store['obj1']

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781
...,...
95,0.795253
96,0.118110
97,-0.748532
98,0.584970


* HDFStore⽀持两种存储模式， 'fixed'和'table'。后者通常会更
慢，但是⽀持使⽤特殊语法进⾏查询操作：

In [110]:
store.put('obj2', frame, format='table')
store.select('obj2', where=['index >= 10 and index <= 15'])
store.close()

In [111]:
frame.to_hdf('mydata.h5', 'obj3', format='table')
pd.read_hdf('mydata.h5', 'obj3', where=['index < 5'])

,a
0,-0.204708
1,0.478943
2,-0.519439
3,-0.555730
4,1.965781


In [115]:
import os
os.remove('mydata.h5')

### 读取Microsoft Excel⽂件 Reading Microsoft Excel Files

* pandas的ExcelFile类或pandas.read_excel函数⽀持读取存储在
Excel 2003（或更⾼版本）中的表格型数据。这两个⼯具分别使
⽤扩展包xlrd和openpyxl读取XLS和XLSX⽂件。你可以⽤pip或
conda安装它们。

* 要使⽤ExcelFile，通过传递xls或xlsx路径创建⼀个实例：

In [118]:
xlsx = pd.ExcelFile('G:/test/pydata-book/examples/ex1.xlsx')

* 存储在表单中的数据可以read_excel读取到DataFrame（原书这
⾥写的是⽤parse解析，但代码中⽤的是read_excel，是个笔
误：只换了代码，没有改⽂字）：

In [119]:
pd.read_excel(xlsx, 'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 如果要读取⼀个⽂件中的多个表单，创建ExcelFile会更快，但你
也可以将⽂件名传递到pandas.read_excel：

In [120]:
frame = pd.read_excel('G:/test/pydata-book/examples/ex1.xlsx', 'Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 如果要将pandas数据写⼊为Excel格式，你必须⾸先创建⼀个
ExcelWriter，然后使⽤pandas对象的to_excel⽅法将数据写⼊到
其中

In [122]:
writer = pd.ExcelWriter('G:/test/pydata-book/examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

* 你还可以不使⽤ExcelWriter，⽽是传递⽂件的路径到to_excel：

In [123]:
frame.to_excel('G:/test/pydata-book/examples/ex2.xlsx')

In [125]:
!type G:\test\pydata-book\examples\ex2.xlsx

PK      ! u犉ぢ  �     xl/worksheets/sheet1.xml崝M徾 嗭�坽兛抣#郢�:Q{═m?钅hm皜M�_0僭�zc辻gx仪逤徫 d荵喢U�皰Wk2�笄胥FRQV褳3瑞慀1�恀竫�-�B��n�w勅矃伿侀L团@�EC�(�VS研�(6d�枚肗麿^譣	/�`�6蠸ヱe蹗鏸臻湙	�3�顜	&y:睃珒嫓瓚ⅶ镄C�犚骳d;q�j抇�楻波=NP�獱�o絲釛闲5�襇址�
猦�
~Abj.Gj�*躮4gi慕Q�湲4翮<H蒠oY^螷Gx�(枎柁qX:鈡莙镠n�'笉�苮姰鍨鰬�9騺��9@9毩��氨6灹:X�豬ga�簈`�簎`��嗬�3貚6衤&3X绱錾�~r`�:煛甘卫嚝齑諻�镏�%碂{�
|ア闃D=院&Xm1�.Lk徘i灯枘曗脅躁� 剦糸詼�骼\观��PK      ! �j%H  &     xl/workbook.xml峇薔�0俭洲ij#Z5┠KTB�Di�&�4V;�隐=隩)p愦3爿盐z�:6妢�u诣扞ui勗�>6徸7罍鏩pe4鎝B�鈐�{�4胬h^�j镗E共茊粔iQ覭el�=Q粡\k�W#鶩EigQ氓喅寐�敲T�,褶擼冓烳,*頸{W酥A爆ぢ�9鉳��Z�█)铧儛ESⅵ�?傢谯N狜f�⑩蛲2�飻咝j�;�+潶i:C譜b飤�e菨月�9�S红id�X?酀�&!嬬�	寰�9坛,嫜/麽~cez�pB�隁�'l拃]媎p荍甁J市楴g�X�)uG讷~6|0Cc掆PK      ! �鎔ú   7     xl/sharedStrings.xmle螷N1�=�埣�XT%��'(3頛つ炂灃逓 剟买�鼟葆砫s�*壣糜n �4驍h鲳~:>線�4盘��(pNDM%癜ó�质竊壊�︰蝄K�雔e�'Y礵�<{[b"0#o�趰嵰e梅�湦�4Dg58�~啵嚤嚛噦"q茷虣{紂��滪O焟焽/PK      ! ^骇觲       docProps/app.xml潚罭�0E鱸E�=uR!鬞9F█X餌�Xg襒8跺�柉荌諓+埠3su}2陡诘6� Ⅰ甦�,g8�+愣%{谲��c抮暡轆肾�霬瀴U�"�,%8

## 6.3 Web APIs交互 Interacting with Web APIs

* 许多⽹站都有⼀些通过JSON或其他格式提供数据的公共API。
通过Python访问这些API的办法有不少。⼀个简单易⽤的办法
（推荐）是requests包（http://docs.python-requests.org）。
为了搜索最新的30个GitHub上的pandas主题，我们可以发⼀个
HTTP GET请求，使⽤requests扩展库：

In [126]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

* 响应对象的json⽅法会返回⼀个包含被解析过的JSON字典，加
载到⼀个Python对象中：

In [127]:
data = resp.json()
data[0]['title']

'Grouping, then resampling empty DataFrame leads to it losing column names and index'

* data中的每个元素都是⼀个包含所有GitHub主题⻚数据（不包含
评论）的字典。我们可以直接传递数据到DataFrame，并提取感
兴趣的字段：

In [129]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,26411,"Grouping, then resampling empty DataFrame lead...",[],open
1,26410,"API: getting the ""densified"" values of a Categ...","[{'default': False, 'color': 'AD7FA8', 'url': ...",open
2,26409,DEPR: deprecate get_values,[],open
3,26408,BUG: MultiIndex not dropping nan level and inv...,[],open
4,26407,SparseArrays backed by other Extension array/d...,"[{'default': False, 'color': '6138b5', 'url': ...",open
...,...,...,...,...
25,26384,Remove Deprecated Azure CI Step,[],open
26,26383,Prefixed batch script commands with call,[],open
27,26381,BUG: error calculating BusinessHourMixin.apply...,"[{'default': False, 'color': 'e10c02', 'url': ...",open
28,26380,ENH: Implementing NEP 18's __array_function__,[],open


* 花费⼀些精⼒，你就可以创建⼀些更⾼级的常⻅的Web API的接
⼝，返回DataFrame对象，⽅便进⾏分析。

##  6.4 数据库交互 Interacting with Databases

* 在商业场景下，⼤多数数据可能不是存储在⽂本或Excel⽂件
中。基于SQL的关系型数据库（如SQL Server、 PostgreSQL和
MySQL等）使⽤⾮常⼴泛，其它⼀些数据库也很流⾏。数据库
的选择通常取决于性能、数据完整性以及应⽤程序的伸缩性需
求。
将数据从SQL加载到DataFrame的过程很简单，此外pandas还
有⼀些能够简化该过程的函数。例如，我将使⽤SQLite数据库
（通过Python内置的sqlite3驱动器）：

In [131]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)
con.commit()

* 然后插⼊⼏⾏数据：

In [132]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

* 从表中选取数据时，⼤部分Python SQL驱动器（PyODBC、
psycopg2、 MySQLdb、 pymssql等）都会返回⼀个元组列表：

In [133]:
cursor = con.execute('select * from test')
rowds = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

* 你可以将这个元组列表传给DataFrame构造器，但还需要列名
（位于光标的description属性中）：

In [134]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


* 这种数据规整操作相当多，你肯定不想每查⼀次数据库就重写⼀
次。 SQLAlchemy项⽬是⼀个流⾏的Python SQL⼯具，它抽象出
了SQL数据库中的许多常⻅差异。 pandas有⼀个read_sql函数，
可以让你轻松的从SQLAlchemy连接读取数据。这⾥，我们⽤
SQLAlchemy连接SQLite数据库，并从之前创建的表读取数据：

In [135]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [136]:
!rm mydata.sqlite

'rm' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## Conclusion